In [1]:
# basic imports
import numpy as np
import IPython
import librosa

import torch

from music2latent.hparams import hparams
from music2latent.hparams_inference import *

device = "mps"

/Users/jasperrr/miniforge3/envs/pt25/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

from music2latent.export import ScriptedUNet
# filepath = os.path.abspath(__file__)
# lib_root = os.path.dirname(filepath)

# load_path_inference_default = os.path.join(lib_root, 'models/music2latent.pt')

load_path_inference_default = 'models/music2latent.pt'

In [ ]:
gen = UNet().to(self.device)